## Working with Geopandas

In [2]:
import geopandas as gpd

### Reading Spatial Data

In [6]:
import os
home_dir = os.path.expanduser('~')
data_pkg_path = 'Downloads/python_foundation/'
filename = 'karnataka.gpkg'
path = os.path.join(home_dir, data_pkg_path, filename)

In [11]:
roads_gdf = gpd.read_file(path, layer='karnataka_major_roads')
roads_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 42475 entries, 0 to 42474
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   osm_id    42475 non-null  object  
 1   code      42475 non-null  int64   
 2   fclass    42475 non-null  object  
 3   name      6859 non-null   object  
 4   ref       7173 non-null   object  
 5   oneway    42475 non-null  object  
 6   maxspeed  42475 non-null  int64   
 7   layer     42475 non-null  int64   
 8   bridge    42475 non-null  object  
 9   tunnel    42475 non-null  object  
 10  geometry  42475 non-null  geometry
dtypes: geometry(1), int64(3), object(7)
memory usage: 3.6+ MB


In [28]:
filtered = roads_gdf[roads_gdf['ref'].str.contains('^NH') == True]
filtered.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
15,8684837,5112,trunk,Bengaluru - Mangaluru Road,NH373,F,0,0,F,F,"MULTILINESTRING ((76.10024 13.00326, 76.09950 ..."
24,9951034,5112,trunk,None,NH948,B,50,0,F,F,"MULTILINESTRING ((77.16472 12.24774, 77.16416 ..."
50,22838314,5112,trunk,Solapur-Mangalore Highway,NH169,B,0,0,F,F,"MULTILINESTRING ((74.86387 12.88387, 74.86419 ..."
51,22838318,5112,trunk,None,NH66,B,0,0,F,F,"MULTILINESTRING ((74.78756 13.09142, 74.78744 ..."
53,22838320,5112,trunk,None,NH66,B,0,0,F,F,"MULTILINESTRING ((74.78785 13.09039, 74.78774 ..."


### Working with Projections

In [22]:
roads_reprojected = filtered.to_crs('EPSG:32643')

In [48]:
roads_reprojected['length'] = roads_reprojected['geometry'].length
sum(roads_reprojected['length'])

8616166.058402948

### Spatial joins

In [27]:
districts_gdf = gpd.read_file(path, layer='karnataka_districts')
districts_gdf.head()

,DISTRICT,ST_NM,ST_CEN_CD,DT_CEN_CD,censuscode,geometry
0,Bagalkot,Karnataka,29,2,556,"MULTIPOLYGON (((76.24100 16.16531, 76.23538 16..."
1,Bangalore Rural,Karnataka,29,29,583,"MULTIPOLYGON (((77.38701 13.50002, 77.40099 13..."
2,Bangalore,Karnataka,29,18,572,"MULTIPOLYGON (((77.83549 12.86809, 77.83213 12..."
3,Belgaum,Karnataka,29,1,555,"MULTIPOLYGON (((75.02647 16.93264, 75.02827 16..."
4,Bellary,Karnataka,29,11,565,"MULTIPOLYGON (((77.15757 15.13706, 77.15887 15..."


In [49]:
districts_reprojected = districts_gdf.to_crs('EPSG:32643')

In [50]:
joined = gpd.sjoin(roads_reprojected, districts_reprojected, how='left', op='intersects')
joined.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry,lengths,length,index_right,DISTRICT,ST_NM,ST_CEN_CD,DT_CEN_CD,censuscode
15,8684837,5112,trunk,Bengaluru - Mangaluru Road,NH373,F,0,0,F,F,"MULTILINESTRING ((619317.005 1437753.696, 6192...",2352.939132,2352.939132,16,Hassan,Karnataka,29,20,574
24,9951034,5112,trunk,None,NH948,B,50,0,F,F,"MULTILINESTRING ((735483.761 1354892.067, 7354...",12876.925507,12876.925507,7,Chamrajnagar,Karnataka,29,24,578
50,22838314,5112,trunk,Solapur-Mangalore Highway,NH169,B,0,0,F,F,"MULTILINESTRING ((485231.162 1424297.721, 4852...",822.021433,822.021433,11,Dakshina Kannada,Karnataka,29,21,575
51,22838318,5112,trunk,None,NH66,B,0,0,F,F,"MULTILINESTRING ((476971.413 1447255.675, 4769...",651.099637,651.099637,11,Dakshina Kannada,Karnataka,29,21,575
51,22838318,5112,trunk,None,NH66,B,0,0,F,F,"MULTILINESTRING ((476971.413 1447255.675, 4769...",651.099637,651.099637,27,Udupi,Karnataka,29,15,569


In [54]:
joined.groupby('DISTRICT')['length'].sum()/1000

DISTRICT
Bagalkot            261.265850
Bangalore           353.679374
Bangalore Rural     324.781909
Belgaum             576.525403
Bellary             314.952985
Bidar               264.335734
Bijapur             424.619975
Chamrajnagar        228.044696
Chikkaballapura     231.510605
Chikmagalur         381.696732
Chitradurga         542.485675
Dakshina Kannada    320.609502
Davanagere          208.278605
Dharwad             346.452273
Gadag               119.149658
Gulbarga            311.154261
Hassan              490.893487
Haveri              310.476062
Kodagu               81.489449
Kolar               258.951946
Koppal              292.843730
Mandya              375.663815
Mysore              363.930769
Raichur             168.202315
Ramanagara          232.078048
Shimoga             500.915495
Tumkur              700.602804
Udupi               281.769635
Uttara Kannada      427.239110
Yadgir              144.962895
Name: length, dtype: float64